#Тестовое Яндекс.Картинки

##Подготовка данных

Загрузим архив с данными с GitHub.

In [2]:
!git clone https://github.com/KirGo-91/yandex-data.git

Cloning into 'yandex-data'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), 17.10 MiB | 21.56 MiB/s, done.


In [1]:
# загрузим библиотеки

from zipfile import ZipFile
import pandas as pd
import plotly.express as px
import numpy as np
from scipy.stats import *

In [3]:
with ZipFile('yandex-data/data.zip', 'r') as zip:
  zip.extractall(path='unzipped')

Пример кода, если необходимо открыть архив с паролем:
```python
from zipfile import ZipFile
import threading

# Скачиваем файл
!wget https://www.cs.ucr.edu/~eamonn/time_series_data/UCR_TS_Archive_2015.zip

# Создаем функцию для разархивации архива с паролем
def uncompress(path, password):
  with ZipFile(path, "r") as zip:
    zip.extractall(path="uncompressed", pwd=password.encode("utf-8"))

file_name = "UCR_TS_Archive_2015.zip"

# Запускаем функцию в отдельный поток, чтобы она работала в фоне и не мешала нам, т.к. данных очень много
threading.Thread(target=uncompress, args=(file_name, 'attempttoclassify')).start()
```

In [4]:
df = pd.read_json('unzipped/data.json')
df.head()

,query,ts,platform
0,порно фото,1631786697,touch
1,малевич картины,1631806465,desktop
2,Секс,1631781583,touch
3,с днём рождения лена,1631771563,touch
4,зверополис порно,1631787599,touch


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203824 entries, 0 to 1203823
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   query     1203824 non-null  object
 1   ts        1203824 non-null  int64 
 2   platform  1203824 non-null  object
dtypes: int64(1), object(2)
memory usage: 27.6+ MB


In [6]:
# посмотрим с каких платформ заходили на сайт

df['platform'].value_counts(normalize=True)

,proportion
platform,
touch,0.658118
desktop,0.341882


In [7]:
df['query'] = df['query'].str.lower()
df['ts'] = pd.to_datetime(df['ts'], unit='s')

In [8]:
print(f'Минимальная дата - {min(df["ts"])}, максимальная дата - {max(df["ts"])}')

Минимальная дата - 2021-08-31 21:00:00, максимальная дата - 2021-09-21 20:59:59


Необходимо привести время к московской временной зоне и устранить трехчасовое отставание.

In [9]:
df['ts'] = df['ts'] + pd.Timedelta('03:00:00')

Добавим столбец с датой

In [10]:
df['date'] = df['ts'].dt.strftime('%Y-%m-%d')

In [11]:
df.sample(5)

,query,ts,platform,date
620598,короткая стрижка мужская,2021-09-04 11:20:27,touch,2021-09-04
1050706,красивые картинки на рабочий стол,2021-09-09 10:46:58,desktop,2021-09-09
390840,фотообои на стену,2021-09-07 17:00:05,touch,2021-09-07
1184945,знаки пдд,2021-09-14 09:42:55,touch,2021-09-14
543023,перу,2021-09-11 22:40:15,desktop,2021-09-11


In [12]:
groupped = (
    df
    .groupby('date', as_index=False)['query']
    .agg(cnt='count')
)

fig = px.line(
    groupped,
    x='date', y='cnt',
    markers=True,
    line_shape='spline',
    title='Динамика запросов в Яндекс.Картинки',
    template='plotly_dark',
    labels={'date': 'Дата', 'cnt': 'Кол-во'}
)

fig.show()

##Задание 1

Определить ТОП-10 запросов по каждой платформе.

In [21]:
# первый вариант
top = (
    df
    .groupby(['platform', 'query'])
    #группировка с мультииндексом
    #.agg({'count'})
    #.droplevel(1, axis='columns')
    ['ts']
    .agg(cnt='count')
    .sort_values('cnt', ascending=False)
    .groupby('platform')
    .head(10)
    .sort_values(['platform', 'cnt'], ascending=False)
)
top

cnt
platform query                                 
touch    порно                            10076
         секс                              8262
         с днём рождения женщине           4924
         хентай                            4194
         с днём рождения                   4123
         с днём рождения мужчине           3669
         с днем рождения                   3315
         xxx                               3249
         xnxx                              2767
         доброе утро                       2342
desktop  календарь 2021                    2810
         таблица менделеева                2659
         картинки                          1720
         порно                             1408
         английский алфавит                1352
         обои на рабочий стол              1143
         одноклассники (социальная сеть)   1116
         секс                               947
         алфавит                            923
         таблица квадратов                  883

In [22]:
# второй вариант
top = pd.concat([
    df[df['platform']=='touch']
    .groupby(['platform', 'query'])
    ['ts']
    .agg(cnt='count')
    .nlargest(10, 'cnt'),
    df[df['platform']=='desktop']
    .groupby(['platform', 'query'])
    ['ts']
    .agg(cnt='count')
    .nlargest(10, 'cnt')
])
top

cnt
platform query                                 
touch    порно                            10076
         секс                              8262
         с днём рождения женщине           4924
         хентай                            4194
         с днём рождения                   4123
         с днём рождения мужчине           3669
         с днем рождения                   3315
         xxx                               3249
         xnxx                              2767
         доброе утро                       2342
desktop  календарь 2021                    2810
         таблица менделеева                2659
         картинки                          1720
         порно                             1408
         английский алфавит                1352
         обои на рабочий стол              1143
         одноклассники (социальная сеть)   1116
         секс                               947
         алфавит                            923
         таблица квадратов                  883

##Задание 2

Показать динамику самых популярных запросов в течение недели.

In [23]:
# добавим столбец с днем недели. Понедельник это цифра 1.
df['weekday'] = df['ts'].dt.dayofweek.map({0: '1 Пн', 1: '2 Вт', 2: '3 Ср', 3: '4 Чт', 4: '5 Пт', 5: '6 Сб', 6: '7 Вс'})

In [24]:
weekday_distribution = (
    df
    .groupby('weekday', as_index=False)
    ['ts']
    .agg(cnt='count')
)

In [26]:
fig = px.bar(
    weekday_distribution,
    x='weekday', y='cnt',
    text_auto=True,
    title='Распределение запросов по дням недели',
    labels={'weekday': 'День недели', 'cnt': 'Кол-во'}
)

fig.show()

##Задание 3

Показать динамику популярных запросов в течение дня.

In [27]:
df['hour'] = df['ts'].dt.hour

In [28]:
# возьмем три самых популярных запроса в каждый час и, используя свойство множества,
# оставим только уникальные, которые встречаются в течение суток
most_frequent_query = set()

for h in df['hour'].unique():
  most_frequent_query.update(
      df
      [df['hour'] == h]
      .groupby('query', as_index=False)
      ['ts']
      .agg(cnt='count')
      .sort_values('cnt', ascending=False)
      .head(3)
      ['query']
      .tolist()
  )

In [29]:
most_frequent_query

{'xxx',
 'доброе утро',
 'календарь 2021',
 'порно',
 'с днём рождения',
 'с днём рождения женщине',
 'секс',
 'таблица менделеева',
 'хентай'}

In [30]:
# построим два графика: для десктопной и мобильной платформ
fig = px.line(
    df
    [df['query'].isin(most_frequent_query)]
    .groupby(['platform', 'query', 'hour'], as_index=False)
    ['ts']
    .agg(cnt='count')
    ,
    x='hour',
    y='cnt',
    color='query',
    facet_col='platform'
)

fig.update_yaxes(matches=None)

fig.show()

##Задача 4

Найти запросы, которые сильнее всего отличаются между платформами. Проверить статистически значима ли разница.

In [31]:
# сделаем сводную таблицу, где сравнивается кол-во запросов между платформами
platform_comparison = (
    df
    .groupby(['platform', 'query'], as_index=False)
    ['ts']
    .agg(cnt='count')
    .pivot_table(
        index = 'query'
        , columns = 'platform'
        , values = 'cnt'
        , fill_value = 0
    )
)

platform_comparison

platform,desktop,touch
query,,
+18,11.0,34.0
+18 video,2.0,24.0
002,4.0,15.0
02 аниме,34.0,30.0
02 милый во франсе,7.0,23.0
...,...,...
سكس,6.0,77.0
幼女,2.0,14.0
视频,0.0,69.0


In [32]:
# посмотрим на распределение данных
platform_comparison.describe()

platform,desktop,touch
count,24026.000000,24026.000000
mean,17.130026,32.975027
std,48.088618,130.368264
min,0.000000,0.000000
25%,4.000000,11.000000
50%,9.000000,17.000000
75%,17.000000,29.000000
max,2810.000000,10076.000000


Отфильтруем таблицу от нулевых и слишком низкочастотных запросов. Оставим только те, количество которых превышает 15.

In [33]:
platform_comparison = platform_comparison[(platform_comparison['desktop'] > 15) & (platform_comparison['touch'] > 15 )]

Определим разницу между запросами на платформах.

In [34]:
platform_comparison['diff'] = platform_comparison['desktop'] - platform_comparison['touch']

/tmp/ipython-input-34-571728271.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



В таком случае, чем больше отрицательная разница, тем больше запросов было с мобильных устройств. И наоборот, чем больше положительная разница, тем больше запросов было с десктопов.  

Используя квантили, оставим в нашем датафрейми только самые отличающиеся запросы. Возьмем первый и девяносто девятый квантили.

In [35]:
platform_comparison = platform_comparison[(platform_comparison['diff'] < np.quantile(platform_comparison['diff'], 0.01))
                                          | (platform_comparison['diff'] > np.quantile(platform_comparison['diff'], 0.99))]

Прежде чем проверить статистическую значимость, необходимо проверить данные на нормальность. Сформируем гипотезы: H0 - данные распределены нормально, альтернативная H1 - данные распределены не нормально. `P-value` примем 0.05.

In [36]:
#Проверим двумя вариантами тестов
for i in platform_comparison[['desktop', 'touch']]:
  print(i, '\n', shapiro(platform_comparison[i]), '\n', normaltest(platform_comparison[i]), '\n')

desktop 
 ShapiroResult(statistic=np.float64(0.6090126229737667), pvalue=np.float64(1.9002868441654518e-14)) 
 NormaltestResult(statistic=np.float64(105.11176350962829), pvalue=np.float64(1.497168090517597e-23)) 

touch 
 ShapiroResult(statistic=np.float64(0.6469090870403142), pvalue=np.float64(1.050178155828377e-13)) 
 NormaltestResult(statistic=np.float64(90.37827649097886), pvalue=np.float64(2.3692266985960728e-20)) 



Оба теста показывают, что необходимо отвергнуть нулевую гипотезу в пользу альтернативной.   
Значит данные распределены не нормально. В таком случае используют непараметрические тесты.  
Для определения статистической значимости в разнице между количествами запросов, сравним дисперсии двух выборок (десктопной и мобильной). Сравнивать будем с помощью критерия Левена. Сформируем гипотезы: H0 - дисперсии равны, альтернативная H1 - дисперсии не равны. `P-value` примем 0.05.

In [37]:
res = levene(platform_comparison['desktop'], platform_comparison['touch'])
res

LeveneResult(statistic=np.float64(24.186133154134186), pvalue=np.float64(1.9159943366210286e-06))

`P-value` крайне мал, значит отвергаем нулевую гипотезу в пользу альтернативной. Из этого следует вывод, что разница в количестве запросов между платформами статистически значима.

In [38]:
platform_comparison.shape

(94, 3)

В таблице осталось 94 запроса. Выделим топ-5 запросов для каждой платформы.

In [39]:
most_different = pd.concat([
    platform_comparison
    .sort_values('diff')
    .head(5),
    platform_comparison
    .sort_values('diff', ascending=False)
    .head(5)
])

In [40]:
most_different['superior'] = (
    most_different['diff']
    .apply(lambda x: 'touch ↗' if x < 0 else 'desktop ↗')
)

In [41]:
most_different

platform,desktop,touch,diff,superior
query,,,,
порно,1408.0,10076.0,-8668.0,touch ↗
секс,947.0,8262.0,-7315.0,touch ↗
с днём рождения женщине,503.0,4924.0,-4421.0,touch ↗
с днём рождения,297.0,4123.0,-3826.0,touch ↗
хентай,821.0,4194.0,-3373.0,touch ↗
обои на рабочий стол,1143.0,102.0,1041.0,desktop ↗
календарь на 2022 год с праздниками и выходными,757.0,41.0,716.0,desktop ↗
youtube,838.0,151.0,687.0,desktop ↗
раскраски,704.0,61.0,643.0,desktop ↗
